## Model Training Trial-runs 

In [1]:
import os 

In [2]:
os.chdir(r"C:\Users\Junior\OneDrive\Desktop\Heart-Attack-Prediction-Model")

## Trial-runs for entity_config 

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path 
    model_name: str 
    alpha: float
    l1_ratio: float
    target_column: str

Traial-runs for ConfigManager

In [4]:
#importing all project paths and modules necessary for project configurations 
from heartAttack.constants import  *
from heartAttack.utils.common import read_yaml, create_directories

In [5]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        
        selected_schema_filepath = SELECTED_SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.selected_schema = read_yaml(selected_schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_model_training_config(self) -> ModelTrainerConfig:
        config = self.config.model_training
        params = self.params.ELASTICNET
        selected_schema = self.selected_schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_training_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name=config.model_name, 
            alpha= params.alpha,
            l1_ratio= params.l1_ratio,
            target_column=selected_schema.name
        )
        return model_training_config

In [6]:
import pandas as pd 
import os 
import joblib
from sklearn.linear_model import ElasticNet
from heartAttack import logger

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [8]:
#pipeline 
try:
    config = ConfigurationManager()
    model_training_config =config.get_model_training_config()
    model_training = ModelTrainer(config=model_training_config)
    model_training.train()
except Exception as e:
    raise e

[2025-07-29 23:17:31,075: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-07-29 23:17:31,079: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-07-29 23:17:31,082: INFO: common: yaml_file: selected_schema.yaml loaded successfully]
[2025-07-29 23:17:31,086: INFO: common: Created directory at: artifacts]
[2025-07-29 23:17:31,089: INFO: common: Created directory at: artifacts/model_training]
